In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

# Fitting Linear Models

In this notebook we briefly review the normal equations, introduce the scikit-learn framework, evaluation metrics, and describe methods to visualize model fit.

## Toy Data set

To enable easy visualization of the model fitting process we will use a simple synthetic data set.

In [ ]:
data = pd.read_csv("data/synthetic_data.csv")
data.head()

We can visualize the data in three dimensions:

In [ ]:
data_scatter = go.Scatter3d(x=data["X0"], y=data["X1"], z=data["Y"], 
                            mode="markers",
                            marker=dict(size=2))
layout = dict(margin=dict(l=0, r=0, t=0, b=0), 
              height=600,
              scene = dict(xaxis_title='X0', yaxis_title='X1', zaxis_title='Y'))
go.Figure([data_scatter], layout)

## Fitting an Ordinary Least Squares Model

Given a model of the form: 

$$
\hat{\mathbb{Y}} = f_\theta(\mathbb{X}) = \mathbb{X} \theta
$$

and taking the average squared loss over our data:

$$
R(\theta) = \frac{1}{n}\sum_{i=1}^n \left(\mathbb{Y}_i - (\mathbb{X}\theta)_i\right)^2
$$

In lecture, we showed that the $\hat{\theta}$ that minimizes this loss:

$$
\hat{\theta} = \arg\min_\theta R(\theta)
$$

is given by the solution to the normal equations:

$$
 \hat{\theta} = \left( \mathbb{X}^T \mathbb{X} \right)^{-1} \mathbb{X}^T \mathbb{Y}
$$


We can directly implement this expression using standard linear algebra routines:

In [ ]:
from numpy.linalg import inv
def ordinary_least_squares(X, Y):
    return inv(X.T @ X) @ X.T @ Y

In [ ]:
theta_hat = ordinary_least_squares(data[["X0", "X1"]], data[["Y"]])
theta_hat

A more efficient way to solve the normal equations is using the `solve` function to solve the linear systems:

$$
\mathbb{X}^T \mathbb{X} \theta = \mathbb{X}^T \mathbb{Y}
$$

can be simplified to:

$$
A \theta = b
$$

where $A=\mathbb{X}^T \mathbb{X}$ and $b=\mathbb{X}^T \mathbb{Y}$:

In [ ]:
from numpy.linalg import solve
def ordinary_least_squares(X, Y):
    return solve(X.T @ X, X.T @ Y)

In [ ]:
theta_hat = ordinary_least_squares(data[["X0", "X1"]], data[["Y"]])
theta_hat

Notice that this second implementation produces a numpy array?  This is because Pandas actually implements inversion but the solve routine is entirely in numpy.

## Making A Prediction

We can use our $\hat{\theta}$ to make predictions:

$$
\hat{y} = X \hat{\theta}
$$

In [ ]:
data["Yhat"] = data[["X0", "X1"]] @ theta_hat
data

How good are our predictions? We can plot $Y$ vs $\hat{Y}$

In [ ]:
fig = px.scatter(data, x="Yhat", y="Y")
fig.add_trace(go.Scatter(x=[-5,5], y=[-5,5], name="y=yhat"))
fig.update_layout(xaxis_title ="Yhat")

We can also plot the residual distribution.

In [ ]:
data['residuals'] = data['Y'] - data['Yhat']
data

In [ ]:
px.histogram(data, x='residuals', nbins=100)

## Visualizing the Model

For the synthetic data set we can visualize the model in three dimensions.  To do this we will use the following plotting function that at evenly space grid points in for `X0` and `X1`.

In [ ]:
def make_surface(f, X, grid_points = 30, name="Linear Model"):
    u = np.linspace(X[:,0].min(),X[:,0].max(), grid_points)
    v = np.linspace(X[:,1].min(),X[:,1].max(), grid_points)
    xu, xv = np.meshgrid(u,v)
    X = np.vstack((xu.flatten(),xv.flatten())).transpose()
    z = f(X)
    return go.Surface(x=xu, y=xv, z=z.reshape(xu.shape), opacity=0.8, name=name,  showscale=False)

Plotting the data and the plane

In [ ]:
simple_plane = make_surface(
    lambda X: X @ theta_hat, 
    data[["X0", "X1"]].to_numpy(), 
    grid_points=5,
    name="plane")
go.Figure([data_scatter, simple_plane], layout)

Notice that the plane is constrained to pass through the origin.  To fix this, we will need to add a constant term to the model.  However, to simplify the process let's switch to using the scikit-learn python library for our modeling.

## Introducing Scikit-learn

In this class, we introduce the normal equations as well as several other algorithms to provide some insight behind how these techniques work and perhaps more importantly how they fail.  However, in practice you will seldom need to implement the core algorithms and will instead use various machine learning software packages.  In this class, we will focus on the widely used scikit-learn package. 

Scikit-learn, or as the cool kids call it sklearn (pronounced s-k-learn), is an large package of useful machine learning algorithms. For this lecture, we will use the `LinearRegression` model in the [`linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) module.  The fact that there is an entire module with many different models within the `linear_model` module might suggest that we have a lot to cover still (we do!).  

**What you should know about `sklearn` models:**

1. Models are created by first building an instance of the model:
```python
model = SuperCoolModelType(args)
```
1. You then fit the model by calling the **fit** function passing in data:
```python
model.fit(df[['X1' 'X1']], df[['Y']])
```
1. You then can make predictions by calling **predict**:
```python
model.predict(df2[['X1' 'X1']])
```

The neat part about sklearn is most models behave like this.  So if you want to try a cool new model you just change the class of mode you are using. 


## Using Scikit-learn

We import the `LinearRegression` model

In [ ]:
from sklearn.linear_model import LinearRegression

Create an instance of the model.  This time we will add an intercept term to the model directly.

In [ ]:
model = LinearRegression(fit_intercept=True)

Fit the model by passing it the $X$ and $Y$ data:

In [ ]:
model.fit(data[["X0", "X1"]], data[["Y"]])

In [ ]:
model.coef_

In [ ]:
model.intercept_

Make some predictions and even save them back to the original DataFrame

In [ ]:
data['Yhat_sklearn'] = model.predict(data[["X0", "X1"]])
data

Analyzing the fit again:

In [ ]:
fig = px.scatter(data, x="Yhat_sklearn", y="Y")
fig.add_trace(go.Scatter(x=[0,10], y=[0,10], name="y=yhat"))
fig.update_layout(xaxis_title ="yhat")

We can also plot the residual distribution. 

In [ ]:
data['residuals_sklearn'] = data['Y'] - data['Yhat_sklearn'] 
data

In [ ]:
px.histogram(data['residuals_sklearn'], nbins=100)

In [ ]:
plane_with_bias = make_surface(model.predict, 
    data[["X0", "X1"]].to_numpy(), 
    grid_points=5,
    name="Plane with Bias")
go.Figure([data_scatter, simple_plane, plane_with_bias], layout)

## Computing Error Metrics

As we tune the features in our model it will be important to define some useful error metrics.  

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
print("Mean Squared Error:", 
      mean_squared_error(data["Y"], data["Yhat_sklearn"]))

In [ ]:
print("Mean Absolute Error:", 
      mean_absolute_error(data["Y"], data["Yhat_sklearn"]))

In [ ]:
print("Root Mean Squared Error:", 
      np.sqrt(mean_squared_error(data["Y"], data["Yhat_sklearn"])))

In [ ]:
print("Standard Deviation of Residuals:",  
      np.std(data['residuals_sklearn']))

As we play with the model we might want a standard visualization

In [ ]:
from plotly.subplots import make_subplots

def evaluate_model(predict, grid_points=30):  
    # Compute Y and Yhat
    Y = data["Y"].to_numpy()
    Yhat = predict(data[["X0", "X1"]].to_numpy()).flatten()
    # Compute and print error metrics
    print("Mean Absolute Error:", mean_absolute_error(Y, Yhat))
    print("Root Mean Squared Error:", np.sqrt(mean_squared_error(Y, Yhat)))

    # Make Side by Side Plots
    fig = make_subplots(rows=1, cols=2, 
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig.add_trace(data_scatter, row=1, col=1)
    surf = make_surface(predict, 
                       data[["X0", "X1"]].to_numpy(), 
                       grid_points=grid_points)
    fig.add_trace(surf, row=1, col=1)
    fig.update_layout(layout)
    
    fig.add_trace(go.Scatter(x=Yhat, y=Y, mode="markers"), row=1, col=2)
    ymin = np.min([np.min(Y), np.min(Yhat)])
    ymax = np.max([np.max(Y), np.max(Yhat)])
    fig.add_trace(go.Scatter(x=[ymin,ymax], y=[ymin,ymax], name="y=yhat"))
    fig.update_layout(xaxis_title ="yhat", yaxis_title="Y")
    fig.update_layout(showlegend=False)
    fig.show()

Examining our latest model:

In [ ]:
evaluate_model(model.predict, grid_points=5)

Our first model without the intercept term:

In [ ]:
no_intercept_model = LinearRegression(fit_intercept=False)
no_intercept_model.fit(data[["X0", "X1"]], data[["Y"]])

In [ ]:
evaluate_model(no_intercept_model.predict, grid_points=5)

Examining the above data we see that there is some **periodic** structure as well as some **curvature**. Can we fit this data with a linear model?